In [4]:
import numpy as np
import matplotlib.pyplot as plt
import suite2p
from scanimagetiffio import SITiffIO
#import utils_image from the parent folder
import sys
sys.path.append("..")
from utils_image import UnrotateCropFrame, RegFrame, compute_zpos_sp


In [3]:
#load the zstack tiff and calculate the mean frames for each stack

zstacktiff = '/home/zilong/Desktop/2D2P/Data/183/stack_00002.tif'
ztstackrelog = '/home/zilong/Desktop/2D2P/Data/183/REdata_20230725_160414.txt'

S = SITiffIO()
S.open_tiff_file(zstacktiff, "r")
S.open_rotary_file(ztstackrelog)
S.interp_times()  # might take a while...

#get all frames and angles in S
Array = []; 
for i in range(S.get_n_frames()):
    Array.append(S.get_frame(i+1))
#change Array to a numpy array
Array = np.array(Array)
Angle = S.get_all_theta()

# unrotate each frame in the tiff file with the detected rotation center
unrotcropFrames = UnrotateCropFrame(Array, Angle, rotCenter=[262,259])


num_v=1; num_s=41; num_f=100
# reshape the unrotFrames to a 5D array
unrotcropFrames = unrotcropFrames.reshape(
    num_v, num_s, num_f, unrotcropFrames.shape[1], unrotcropFrames.shape[2]
)
# average across the first and third dimension
meanZstacks = np.mean(unrotcropFrames, axis=(0, 2))

Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183/REdata_20230725_160414.txt
Finished calculating rotations and times.
The rotary encoder file has 17935 timestamps in it.
Counted 4100 frames


In [6]:
def hist_match(source, template):
    """
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image

    Arguments:
    -----------
        source: np.ndarray
            Image to transform; the histogram is computed over the flattened
            array
        template: np.ndarray
            Template image; can have different dimensions to source
    Returns:
    -----------
        matched: np.ndarray
            The transformed output image
    """

    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()

    # get the set of unique pixel values and their corresponding indices and
    # counts
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    # take the cumsum of the counts and normalize by the number of pixels to
    # get the empirical cumulative distribution functions for the source and
    # template images (maps pixel value --> quantile)
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # interpolate linearly to find the pixel values in the template image
    # that correspond most closely to the quantiles in the source image
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

In [7]:
#visualize each frames in the meanZstacks and save it to a folder 
firstframe = meanZstacks[0,:,:]
for i in range(meanZstacks.shape[0]):
    new_frame_i = meanZstacks[i,:,:]
    if i>0:
        frame_i = meanZstacks[i,:,:]
        #histogram matching to the first frame
        new_frame_i = hist_match(frame_i, firstframe)
    plt.imshow(new_frame_i)
    plt.savefig('/home/zilong/Desktop/2D2P/Data/183/DP/meanZstacks/meanZstacks'+str(i+1)+'.png')

NameError: name 'new_frame_i' is not defined